In [1]:
import os
import os.path
import sys
import getopt
import glob
import operator
import numpy as np

def runCMD(cmd):
    val = subprocess.Popen(cmd, shell=True).wait()
    if val == 0:
        pass
    else:
        print ('command failed')
        print (cmd)
        sys.exit(1)

In [2]:
def runMR(id1,id2):
    mranks = bigDict[id1][id2] * bigDict[id2][id1]
    mr = np.round(np.sqrt(mranks), 3)

    line1 = '\t'.join([str(id2), str(mr)])
    if id1 not in mrDict:
        mrDict[id1] = {}
    mrDict[id1][line1] = mr

    line2 = '\t'.join([str(id1), str(mr)])
    if id2 not in mrDict:
        mrDict[id2] = {}
    mrDict[id2][line2] = mr

def usage():
    print('\n  Usage: '+sys.argv[0]+' -i <pcc directory> -o <mr directory> [-k <skipfile>')
    print("    -i|--indir  <DIRECTORY> path to directory of gene PCCs")
    print("    -o|--outdir <DIRECTORY> path to new directory to store gene mutual ranks")
    print("    -k|--skip   <file> path to file containing list of genes to exclude from mutual rank calculation")
    
    
# Read in command line arguments
try:
    options, remainder = getopt.getopt(sys.argv[1:], 'hi:o:k:', ['help', 'indir=', 'outdir=', 'skip=',])
except getopt.GetoptError as err:
    # print help information and exit:
    print(err)  # will print something like "option -a not recognized"
    usage()
    sys.exit(2)

for opt, arg in options:
    if opt in ('-h', '--help'):
        usage()
        sys.exit()
    elif opt in ('-o', '--outdir'):
        outdir = arg
    elif opt in ('-i', '--indir'):
        indir = arg
    elif opt in ('-t', '--skip'):
        skipfile = int(arg)
command = " ".join(sys.argv)

try:
    indir
    outdir
except NameError:
    usage()
    sys.exit(2)
else:
    print("\nCOMMAND:", command, '\n')

if os.path.exists(output_dirname) == False:
    cmd = 'mkdir ' + output_dirname
    runCMD(cmd)
    print('Creating output directory:', outdir, '...\n')
else:
    print('Writing to existing output directory:', outdir, '...\n')


In [3]:
indir = '/depot/jwisecav/data/jwisecav/repo_testing/test_pcc'
outdir = '/depot/jwisecav/data/jwisecav/repo_testing/test_mr'
skipfile = '/depot/jwisecav/data/jwisecav/repo_testing/exclude_genes.txt'
pcc_threshold = -1
weight_threshold = 0.01

In [4]:
skipList = []
try:
    skipfile
except NameError:
    print('Skipping 0 genes...')
else:
    fi = open(skipfile)
    for gene in skipfile:
        gene = gene.rstrip()
        skipList.append(gene)


In [5]:
mrDict = {}
bigDict = {}
idlookup = {}
genelookup = {}
int_count = 0
files = glob.glob(indir + '/*')

for infile in files:
    #print(infile)
    gene = infile.split('/')[-1]
    #print(gene)
    
    int_count = int_count + 1 
    idlookup[gene] = int_count
    genelookup[int_count] = gene


for infile in files:
    #print(infile)
    gene1 = infile.split('/')[-1]
    print("\tReading", gene1)
    id1 = idlookup[gene1]
    count = 0
    if gene1 in skipList:
        #print('\tskipping', gene1)
        continue


    if id1 not in bigDict:
        bigDict[id1] = {}

    
    fi = open(infile)
    for line in fi:
        [gene2, pcc] = line.rstrip().split('\t')
        if gene1 == gene2:
            #print('\tskipping', gene1,gene2)
            continue
        if gene2 in skipList:
            #print('\tskipping', gene2)
            continue

        id2 = idlookup[gene2]
        count += 1 
        bigDict[id1][id2] = count
        
        if id2 not in bigDict:
            continue
        elif id1 in bigDict[id2]:
            runMR(id1,id2)
            del bigDict[id2][id1]
            del bigDict[id1][id2]
            #print('pair found, calculated MR and removed from dictionary')
            continue 

    fi.close()
print('\nREADING COMPLETE!\n\nCalculating MR scores now...\n')


	Reading Sevir.1G005300
	Reading Sevir.1G000200
	Reading Sevir.1G002100
	Reading Sevir.1G000400
	Reading Sevir.1G004900
	Reading Sevir.1G002000
	Reading Sevir.1G004100
	Reading Sevir.1G003100
	Reading Sevir.1G003900
	Reading Sevir.1G000501
	Reading Sevir.1G002300
	Reading Sevir.1G003500
	Reading Sevir.1G005000
	Reading Sevir.1G001300
	Reading Sevir.1G004300
	Reading Sevir.1G004000
	Reading Sevir.1G004800
	Reading Sevir.1G003300
	Reading Sevir.1G004600
	Reading Sevir.1G000100
	Reading Sevir.1G000900
	Reading Sevir.1G002900
	Reading Sevir.1G003550
	Reading Sevir.1G002400
	Reading Sevir.1G002800
	Reading Sevir.1G003600
	Reading Sevir.1G001700
	Reading Sevir.1G002600
	Reading Sevir.1G000600
	Reading Sevir.1G001600
	Reading Sevir.1G003401
	Reading Sevir.1G001900
	Reading Sevir.1G003000
	Reading Sevir.1G004700
	Reading Sevir.1G003200
	Reading Sevir.1G000300
	Reading Sevir.1G001800
	Reading Sevir.1G002700
	Reading Sevir.1G002500
	Reading Sevir.1G001000
	Reading Sevir.1G005200
	Reading Sevir.1

In [8]:
for gid in mrDict:
    gene = genelookup[gid]
    
    pccDict = {}
    pccfile = indir + '/' + gene
    fi = open(pccfile)
    for line in fi:
        [gene2, pcc] = line.rstrip().split('\t')
        pccDict[gene2] = np.round(float(pcc), 3)
    fi.close()
    
    outfile = outdir + '/' + gene
    fo = open(outfile, 'w')
    
    sorted_d = sorted(mrDict[gid].items(), key=operator.itemgetter(1), reverse = False)
    #print(sorted_d)
    for mr in sorted_d:
        [id2, mr] = mr[0].split('\t')
        gene2 = genelookup[int(id2)]
        line = gene2 + '\t' + mr + '\t' + str(pccDict[gene2]) + '\n'
        fo.write(line)
    
    fo.close()